# Wordle Solver

In [25]:
# import functions
from wordle_helpers import *

In [26]:
# help(main)

In [27]:
# Main function that returns a list of possible words
green_letters = 'RE'
green_positions = [2,5,]
yellow_letters = 'I'
yellow_positions = [2,]
bad_letters = 'CANPVOT'

# Main function that returns a list of possible words
words = main(green_letters, green_positions, yellow_letters, yellow_positions, bad_letters)
print(words)

['bribe', 'bride', 'grime']


In [28]:
# Repetitive letters
letters_df = repetitive_letters(wordle_list=words)
print(letters_df.to_string(index=False))

Letters  Count
      B      3
      R      3
      I      3
      E      3
      D      1
      G      1
      M      1


#### Choosing next word

In [29]:
# Main function that returns a list of possible words
green_letters = ''
green_positions = []
yellow_letters = 'IOPTV'
yellow_positions = []
bad_letters = ''

# Main function that returns a list of possible words
words = main(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    bad_letters,
    wordle_list_path="wordle-possible-words",
)
print(words)

['pivot']


In [30]:
# help(score)

In [31]:
df = load_data(file_name='wordle_ranking.csv')
df

,Names,Games_Won
0,Murilo,165
1,Barbara,74
2,Draw,165


In [32]:
# Update score
score_df = score()
score_df

,Names,Games_Won
0,Murilo,165
1,Barbara,74
2,Draw,166


In [33]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo        165
Barbara         74
   Draw        166


In [34]:
# help(reset_score)

#### Set custom score

In [35]:
# help(set_score)

In [36]:
# # Set custom score
# df = set_score(m_score=140, b_score=60, draw_score=141)
# df

#### Reset score

In [37]:
## Reset score
# reset_score()